In [1]:
!pip install iterative-stratification

In [2]:
from fastai.vision.all import *

In [3]:
def random_seed(seed_value, use_cuda): 
    np.random.seed(seed_value)
    torch.manual_seed(seed_value) 
    random.seed(seed_value)
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True 
        torch.backends.cudnn.benchmark = False 

random_seed(3407, True)

In [6]:
train_df = pd.read_csv('/kaggle/input/hackathon-online-cloud-recognition/train.csv')
train_df['id'] = train_df['id'].apply(lambda x : "/kaggle/input/hackathon-online-cloud-recognition/images/train/" + x)

# Prepare submission dataframe
df_pesudo = pd.read_csv('/kaggle/input/cloudconfi/EVA_02con.csv')

# For pesudo
df_pesudo = df_pesudo[df_pesudo['confidence'] > 0.30]
df_pesudo = df_pesudo[['id', 'predict']]
df_pesudo['id'] = df_pesudo['id'].apply(lambda x : "/kaggle/input/hackathon-online-cloud-recognition/images/test/" + x)

In [7]:
new_header = ['id', 'label']
df_pesudo = df_pesudo.rename(columns=dict(zip(df_pesudo.columns, new_header)))
df_pesudo

,id,label
0,/kaggle/input/hackathon-online-cloud-recognition/images/test/3b46f6ebe56ab934da921e184915c03b.jpg,2
1,/kaggle/input/hackathon-online-cloud-recognition/images/test/9f76ca6c0ef7b1190ea35e61755e00ef.jpg,0
2,/kaggle/input/hackathon-online-cloud-recognition/images/test/56ed4b7fcd81e9eaed5124ad06360b2f.jpg,3
3,/kaggle/input/hackathon-online-cloud-recognition/images/test/1415f3f052d1ec4d6d2d37f8e0e5984c.jpg,6
4,/kaggle/input/hackathon-online-cloud-recognition/images/test/85f426446bb6c35ccf51609daa25d1b3.jpg,0
...,...,...
695,/kaggle/input/hackathon-online-cloud-recognition/images/test/8b687a92b94c887c029305a6a4bbb53e.jpg,2
696,/kaggle/input/hackathon-online-cloud-recognition/images/test/0f9520beab229154e0a41ef3b22f2168.jpg,2
697,/kaggle/input/hackathon-online-cloud-recognition/images/test/8bc634f6d55f1477a632e418773e5e2f.jpg,0
698,/kaggle/input/hackathon-online-cloud-recognition/images/test/4e631ad453252807a2bf1739075be571.jpg,1


In [8]:
frames = [train_df, df_pesudo]

train_df = pd.concat(frames, ignore_index=True)

train_df

,id,label
0,/kaggle/input/hackathon-online-cloud-recognition/images/train/f164dae2e2cdd4e5411e93a8dc95d142.jpg,3
1,/kaggle/input/hackathon-online-cloud-recognition/images/train/588083bc2555da85c1db7e8abd9d6922.jpg,1
2,/kaggle/input/hackathon-online-cloud-recognition/images/train/dd20037db2aa28f7b617bbf5f6754470.jpg,2
3,/kaggle/input/hackathon-online-cloud-recognition/images/train/a6d1c3b86279c54afd4e2fdf0b88111a.jpg,2
4,/kaggle/input/hackathon-online-cloud-recognition/images/train/8989be70528a5ff4cfa6d098afb5722e.jpg,1
...,...,...
1601,/kaggle/input/hackathon-online-cloud-recognition/images/test/8b687a92b94c887c029305a6a4bbb53e.jpg,2
1602,/kaggle/input/hackathon-online-cloud-recognition/images/test/0f9520beab229154e0a41ef3b22f2168.jpg,2
1603,/kaggle/input/hackathon-online-cloud-recognition/images/test/8bc634f6d55f1477a632e418773e5e2f.jpg,0
1604,/kaggle/input/hackathon-online-cloud-recognition/images/test/4e631ad453252807a2bf1739075be571.jpg,1


In [ ]:
# strat_kfold = MultilabelStratifiedKFold(n_splits=10, random_state=42, shuffle=True)
# train_df['fold'] = -1
# for i, (_, test_index) in enumerate(strat_kfold.split(train_df.file.values, train_df.iloc[:,1:].values)):
#     train_df.iloc[test_index, -1] = i
# train_df.head()

In [ ]:
# train_df.fold.value_counts().plot.bar();

In [9]:
class AlbumentationsTransform (RandTransform):
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [10]:
import albumentations
from albumentations.pytorch import ToTensorV2
warnings.filterwarnings('ignore')

size = 336
def get_train_aug(): return albumentations.Compose([
            albumentations.Resize(always_apply=False, p=1.0, height=size, width=size, interpolation=0),
#             AutoContrast(),
            #albumentations.PadIfNeeded(always_apply=False, p=1.0, min_height=size, min_width=size, pad_height_divisor=None, pad_width_divisor=None, border_mode=0, value=(255, 255, 255), mask_value=None), #Extra tip, use size that's suitable for the efficentNetwork you are using.
            albumentations.Rotate(limit=(-360, 360)),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
    
#             albumentations.Sharpen(always_apply=False, p=1.0, alpha=(0.2, 0.5), lightness=(0.5, 1.0)),
            #albumentations.GaussianBlur(always_apply=False, p=1.0, blur_limit=(3, 7), sigma_limit=(0.0, 0)),
            #albumentations.ShiftScaleRotate(always_apply=False, p=1.0, shift_limit_x=(-0.06, 0.06), shift_limit_y=(-0.06, 0.06), scale_limit=(-0.1, 0.1), rotate_limit=(-20, 20), interpolation=0, border_mode=0, value=(255, 255, 255), mask_value=None, rotate_method='largest_box'),
            # albumentations.HueSaturationValue(
            #     hue_shift_limit=0.2, 
            #     sat_shift_limit=0.2, 
            #     val_shift_limit=0.2, 
            #     p=0.5
            # ),
            # #albumentations.ColorJitter(contrast=(0.9, 1.1)),
#             albumentations.CLAHE(always_apply=False, p=0.5, clip_limit=(1, 4), tile_grid_size=(8, 8)),
#             albumentations.RandomBrightnessContrast(
#                 brightness_limit=(0,0.2), 
#                 contrast_limit=(0, 0.2), 
#                 p=0.5
#             ),
            #albumentations.RandomGamma(),
            albumentations.ToGray(always_apply=False, p=1.0)
            
           #albumentations.RandomGridShuffle(p=0.5)\
    ], p=1.)



def get_valid_aug(): return albumentations.Compose([
            albumentations.Resize(always_apply=False, p=1.0, height=size, width=size, interpolation=0),
#             albumentations.Sharpen(always_apply=False, p=1.0, alpha=(0.2, 0.5), lightness=(0.5, 1.0))
            #albumentations.PadIfNeeded(always_apply=False, p=1.0, min_height=size, min_width=size, pad_height_divisor=None, pad_width_divisor=None, border_mode=0, value=(255, 255, 255), mask_value=None),
            albumentations.ToGray(always_apply=False, p=1.0),
            #AutoContrast(),
             
            ], p=1.0)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

In [11]:
def get_data():
    return DataBlock(blocks=(ImageBlock,CategoryBlock),
                       get_x=ColReader(0),
                       get_y=ColReader(1),
                       item_tfms=item_tfms,
                   batch_tfms=[Normalize.from_stats(*imagenet_stats)]
                      ).dataloaders(train_df, bs=32)

In [12]:
test_df = pd.read_csv('/kaggle/input/hackathon-online-cloud-recognition/test.csv')
test_df['id'] = test_df['id'].apply(lambda x : "/kaggle/input/hackathon-online-cloud-recognition/images/test/" + x)
tstpng = test_df.copy()
tstpng.head()

,id
0,/kaggle/input/hackathon-online-cloud-recognition/images/test/3b46f6ebe56ab934da921e184915c03b.jpg
1,/kaggle/input/hackathon-online-cloud-recognition/images/test/9f76ca6c0ef7b1190ea35e61755e00ef.jpg
2,/kaggle/input/hackathon-online-cloud-recognition/images/test/56ed4b7fcd81e9eaed5124ad06360b2f.jpg
3,/kaggle/input/hackathon-online-cloud-recognition/images/test/1415f3f052d1ec4d6d2d37f8e0e5984c.jpg
4,/kaggle/input/hackathon-online-cloud-recognition/images/test/85f426446bb6c35ccf51609daa25d1b3.jpg


In [42]:
# del learn
# torch.cuda.empty_cache()

In [13]:
dls = get_data()
learn = cnn_learner(dls, 'eva_large_patch14_336.in22k_ft_in1k', metrics=accuracy, cbs=[SaveModelCallback(monitor='valid_loss')]).to_fp16()
learn.model = torch.nn.DataParallel(learn.model)
learn.fine_tune(20)

epoch,train_loss,valid_loss,accuracy,time
0,1.284372,0.314975,0.872274,03:20


Better model found at epoch 0 with valid_loss value: 0.31497514247894287.


epoch,train_loss,valid_loss,accuracy,time
0,0.453809,0.147037,0.943925,02:53
1,0.381717,0.126193,0.953271,03:09
2,0.280284,0.091910,0.968847,02:51
3,0.245244,0.112857,0.962617,02:51
4,0.207644,0.144355,0.950156,02:51
5,0.205436,0.141199,0.953271,02:52
6,0.166194,0.227765,0.956386,02:57
7,0.151978,0.139491,0.971963,02:54
8,0.123318,0.082784,0.975078,02:55
9,0.112333,0.111191,0.959502,02:57


Better model found at epoch 0 with valid_loss value: 0.14703689515590668.
Better model found at epoch 1 with valid_loss value: 0.1261928528547287.
Better model found at epoch 2 with valid_loss value: 0.09191036224365234.
Better model found at epoch 8 with valid_loss value: 0.08278437703847885.


KeyboardInterrupt: 

In [14]:
import torch.nn.functional as F

tst_dl = learn.dls.test_dl(tstpng)

# get_preds returns probabilities for each class if model's output isn't post-processed
preds, _ = learn.get_preds(dl=tst_dl)

# Apply softmax to obtain probabilities for each class
probabilities = F.softmax(preds, dim=1)

# Calculate confidence as max probability
confidence = probabilities.max(dim=1)[0]

In [15]:
# Prepare submission dataframe
sub = pd.read_csv('/kaggle/input/hackathon-online-cloud-recognition/test.csv')
idxs = preds.argmax(dim=1)
vocab = np.array(dls.vocab)
sub['predict'] = vocab[idxs]

In [16]:
sub.to_csv('EVA_05-pesudo1.csv', index=False)

In [17]:
sub['confidence'] = confidence
sub.to_csv('EVA_05con-pesudo1.csv', index=False)